In [1]:
%%markdown
# References
* https://github.com/normandipalo/faceID_beta/blob/master/faceid_beta.ipynb


# References
* https://github.com/normandipalo/faceID_beta/blob/master/faceid_beta.ipynb

In [6]:
!ls -laFh /data/blogs/keras-faceid-recognition/

total 0
drwxr-xr-x 2 root root 64 Mar 16 21:48 ./
drwxr-xr-x 3 root root 96 Mar 16 21:48 ../


In [8]:
%%markdown
FaceID recreation using face embeddings and RGBD images.
Made by [Norman Di Palo](https://medium.com/@normandipalo), March 2018.

Let''s start by downloading the dataset.

FaceID recreation using face embeddings and RGBD images.
Made by [Norman Di Palo](https://medium.com/@normandipalo), March 2018.

Let''s start by downloading the dataset.

In [3]:
link_list=["http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(151751).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(153054).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(154211).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(160440).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(160931).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(161342).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(163349).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-16)(164248).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-17)(141550).zip", \
          "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-17)(142154).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-17)(142457).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-17)(143016).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(132824).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(133201).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(133846).zip", \
          "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(134239).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(134757).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(140516).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(143345).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(144316).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(145150).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(145623).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(150303).zip", \
          "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(150650).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(151337).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(151650).zip"]
val_list=["http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(152717).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(153532).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(154129).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(154728).zip", "http://vap.aau.dk/wp-content/uploads/VAPRBGD/(2012-05-18)(155357).zip"]


In [7]:
import requests, zipfile, io
for link in link_list:
  r = requests.get(link, stream=True)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall("/data/blogs/keras-faceid-recognition/faceid_train")
for link in val_list:
  r = requests.get(link, stream=True)
  z = zipfile.ZipFile(io.BytesIO(r.content))
  z.extractall("/data/blogs/keras-faceid-recognition/faceid_val")


In [9]:
%%markdown
# Input preprocessing
Here we create some functions that will create the input couple for our model,
both correct and wrong couples. I created functions to have both depth-only
input and RGBD inputs.


# Input preprocessing
Here we create some functions that will create the input couple for our model,
both correct and wrong couples. I created functions to have both depth-only
input and RGBD inputs.

In [10]:
import numpy as np
import glob
import matplotlib.pyplot as plt
from PIL import Image

In [11]:
def create_couple(file_path):
    folder=np.random.choice(glob.glob(file_path + "*"))
    while folder == "datalab":
      folder=np.random.choice(glob.glob(file_path + "*"))
  #  print(folder)
    mat=np.zeros((480,640), dtype='float32')
    i=0
    j=0
    depth_file = np.random.choice(glob.glob(folder + "/*.dat"))
    with open(depth_file) as file:
        for line in file:
            vals = line.split('\t')
            for val in vals:
                if val == "\n": continue 
                if int(val) > 1200 or int(val) == -1: val= 1200
                mat[i][j]=float(int(val))
                j+=1
                j=j%640

            i+=1
        mat = np.asarray(mat)
    mat_small=mat[140:340,220:420]
    mat_small=(mat_small-np.mean(mat_small))/np.max(mat_small)
#    plt.imshow(mat_small)
#    plt.show()
    
    mat2=np.zeros((480,640), dtype='float32')
    i=0
    j=0
    depth_file = np.random.choice(glob.glob(folder + "/*.dat"))
    with open(depth_file) as file:
        for line in file:
            vals = line.split('\t')
            for val in vals:
                if val == "\n": continue 
                if int(val) > 1200 or int(val) == -1: val= 1200
                mat2[i][j]=float(int(val))
                j+=1
                j=j%640

            i+=1
        mat2 = np.asarray(mat2)
    mat2_small=mat2[140:340,220:420]
    mat2_small=(mat2_small-np.mean(mat2_small))/np.max(mat2_small)
#    plt.imshow(mat2_small)
#    plt.show()
    return np.array([mat_small, mat2_small])

In [12]:
print(create_couple("/data/blogs/keras-faceid-recognition/faceid_train/"))

[[[ 0.12369669  0.12369669  0.12369669 ...  0.12369669  0.12369669
    0.12369669]
  [ 0.12369669  0.12369669  0.12369669 ...  0.12369669  0.12369669
    0.12369669]
  [ 0.12369669  0.12369669  0.12369669 ...  0.12369669  0.12369669
    0.12369669]
  ...
  [-0.13796997 -0.13796997 -0.13796997 ... -0.11463664 -0.11463664
   -0.11713663]
  [-0.14046997 -0.14046997 -0.13796997 ... -0.11463664 -0.11713663
   -0.11713663]
  [-0.14213663 -0.14046997 -0.14213663 ... -0.11713663 -0.11880331
   -0.11880331]]

 [[ 0.11525767  0.11525767  0.11525767 ...  0.11525767  0.11525767
    0.11525767]
  [ 0.11525767  0.11525767  0.11525767 ...  0.11525767  0.11525767
    0.11525767]
  [ 0.11525767  0.11525767  0.11525767 ...  0.11525767  0.11525767
    0.11525767]
  ...
  [-0.15224233 -0.15224233 -0.15224233 ... -0.12307566 -0.12307566
   -0.121409  ]
  [-0.15224233 -0.15224233 -0.15224233 ... -0.12557566 -0.12557566
   -0.12307566]
  [-0.153909   -0.153909   -0.153909   ... -0.12724233 -0.12557566
   -0.

In [13]:
def create_couple_rgbd(file_path):
    folder=np.random.choice(glob.glob(file_path + "*"))
    while folder == "datalab":
      folder=np.random.choice(glob.glob(file_path + "*"))
  #  print(folder)
    mat=np.zeros((480,640), dtype='float32')
    i=0
    j=0
    depth_file = np.random.choice(glob.glob(folder + "/*.dat"))
    with open(depth_file) as file:
        for line in file:
            vals = line.split('\t')
            for val in vals:
                if val == "\n": continue    
                if int(val) > 1200 or int(val) == -1: val= 1200
                mat[i][j]=float(int(val))
                j+=1
                j=j%640

            i+=1
        mat = np.asarray(mat)
    mat_small=mat[140:340,220:420]
    img = Image.open(depth_file[:-5] + "c.bmp")
    img.thumbnail((640,480))
    img = np.asarray(img)
    img = img[140:340,220:420]
    mat_small=(mat_small-np.mean(mat_small))/np.max(mat_small)
#    plt.imshow(mat_small)
#    plt.show()
#    plt.imshow(img)
#    plt.show()
    
    
    mat2=np.zeros((480,640), dtype='float32')
    i=0
    j=0
    depth_file = np.random.choice(glob.glob(folder + "/*.dat"))
    with open(depth_file) as file:
        for line in file:
            vals = line.split('\t')
            for val in vals:
                if val == "\n": continue
                if int(val) > 1200 or int(val) == -1: val= 1200
                mat2[i][j]=float(int(val))
                j+=1
                j=j%640

            i+=1
        mat2 = np.asarray(mat2)
    mat2_small=mat2[140:340,220:420]
    img2 = Image.open(depth_file[:-5] + "c.bmp")
    img2.thumbnail((640,480))
    img2 = np.asarray(img2)
    img2 = img2[160:360,240:440]

 #   plt.imshow(img2)
 #   plt.show()
    mat2_small=(mat2_small-np.mean(mat2_small))/np.max(mat2_small)
 #   plt.imshow(mat2_small)
 #   plt.show()
    
    full1 = np.zeros((200,200,4))
    full1[:,:,:3] = img[:,:,:3]
    full1[:,:,3] = mat_small
    
    full2 = np.zeros((200,200,4))
    full2[:,:,:3] = img2[:,:,:3]
    full2[:,:,3] = mat2_small
    return np.array([full1, full2])

In [14]:
create_couple_rgbd("/data/blogs/keras-faceid-recognition/faceid_val/")

array([[[[ 1.77000000e+02,  1.61000000e+02,  1.34000000e+02,
           1.26669824e-01],
         [ 1.77000000e+02,  1.61000000e+02,  1.32000000e+02,
           1.26669824e-01],
         [ 1.77000000e+02,  1.63000000e+02,  1.31000000e+02,
           1.26669824e-01],
         ...,
         [ 1.55000000e+02,  1.48000000e+02,  1.21000000e+02,
           1.26669824e-01],
         [ 1.54000000e+02,  1.47000000e+02,  1.21000000e+02,
           1.26669824e-01],
         [ 1.53000000e+02,  1.46000000e+02,  1.21000000e+02,
           1.26669824e-01]],

        [[ 1.77000000e+02,  1.61000000e+02,  1.32000000e+02,
           1.26669824e-01],
         [ 1.77000000e+02,  1.60000000e+02,  1.32000000e+02,
           1.26669824e-01],
         [ 1.77000000e+02,  1.60000000e+02,  1.29000000e+02,
           1.26669824e-01],
         ...,
         [ 1.55000000e+02,  1.47000000e+02,  1.20000000e+02,
           1.26669824e-01],
         [ 1.52000000e+02,  1.47000000e+02,  1.18000000e+02,
           1.266698

In [15]:
def create_wrong(file_path):
    folder=np.random.choice(glob.glob(file_path + "*"))
    while folder == "datalab":
      folder=np.random.choice(glob.glob(file_path + "*"))    
    mat=np.zeros((480,640), dtype='float32')
    i=0
    j=0
    depth_file = np.random.choice(glob.glob(folder + "/*.dat"))
    with open(depth_file) as file:
        for line in file:
            vals = line.split('\t')
            for val in vals:
                if val == "\n": continue 
                if int(val) > 1200 or int(val) == -1: val= 1200
                mat[i][j]=float(int(val))
                j+=1
                j=j%640

            i+=1
        mat = np.asarray(mat)
    mat_small=mat[140:340,220:420]
    mat_small=(mat_small-np.mean(mat_small))/np.max(mat_small)
 #   plt.imshow(mat_small)
 #   plt.show()
    
    folder2=np.random.choice(glob.glob(file_path + "*"))
    while folder==folder2 or folder2=="datalab": #it activates if it chose the same folder
        folder2=np.random.choice(glob.glob(file_path + "*"))
    mat2=np.zeros((480,640), dtype='float32')
    i=0
    j=0
    depth_file = np.random.choice(glob.glob(folder2 + "/*.dat"))
    with open(depth_file) as file:
        for line in file:
            vals = line.split('\t')
            for val in vals:
                if val == "\n": continue
                if int(val) > 1200 or int(val) == -1: val= 1200
                mat2[i][j]=float(int(val))
                j+=1
                j=j%640

            i+=1
        mat2 = np.asarray(mat2)
    mat2_small=mat2[140:340,220:420]
    mat2_small=(mat2_small-np.mean(mat2_small))/np.max(mat2_small)
 #   plt.imshow(mat2_small)
 #   plt.show()
  
    
    return np.array([mat_small, mat2_small])

In [16]:
create_wrong("/data/blogs/keras-faceid-recognition/faceid_train/")

array([[[ 0.09052429,  0.09052429,  0.09052429, ...,  0.09052429,
          0.09052429,  0.09052429],
        [ 0.09052429,  0.09052429,  0.09052429, ...,  0.09052429,
          0.09052429,  0.09052429],
        [ 0.09052429,  0.09052429,  0.09052429, ...,  0.09052429,
          0.09052429,  0.09052429],
        ...,
        [-0.11197571, -0.11197571, -0.11030904, ...,  0.09052429,
          0.09052429,  0.09052429],
        [-0.11197571, -0.11197571, -0.11197571, ...,  0.09052429,
          0.09052429,  0.09052429],
        [-0.1144757 , -0.1144757 , -0.11197571, ...,  0.09052429,
          0.09052429,  0.09052429]],

       [[ 0.07465963,  0.07465963,  0.07465963, ...,  0.07465963,
          0.07465963,  0.07465963],
        [ 0.07465963,  0.07465963,  0.07465963, ...,  0.07465963,
          0.07465963,  0.07465963],
        [ 0.07465963,  0.07465963,  0.07465963, ...,  0.07465963,
          0.07465963,  0.07465963],
        ...,
        [ 0.07465963,  0.07465963,  0.07465963, ...,  

In [17]:
def create_wrong_rgbd(file_path):
    folder=np.random.choice(glob.glob(file_path + "*"))
    while folder == "datalab":
      folder=np.random.choice(glob.glob(file_path + "*"))    
    mat=np.zeros((480,640), dtype='float32')
    i=0
    j=0
    depth_file = np.random.choice(glob.glob(folder + "/*.dat"))
    with open(depth_file) as file:
        for line in file:
            vals = line.split('\t')
            for val in vals:
                if val == "\n": continue
                if int(val) > 1200 or int(val) == -1: val= 1200
                mat[i][j]=float(int(val))
                j+=1
                j=j%640

            i+=1
        mat = np.asarray(mat)
    mat_small=mat[140:340,220:420]
    img = Image.open(depth_file[:-5] + "c.bmp")
    img.thumbnail((640,480))
    img = np.asarray(img)
    img = img[140:340,220:420]
    mat_small=(mat_small-np.mean(mat_small))/np.max(mat_small)
  #  plt.imshow(img)
  #  plt.show()
  #  plt.imshow(mat_small)
  #  plt.show()
    folder2=np.random.choice(glob.glob(file_path + "*"))
    while folder==folder2 or folder2=="datalab": #it activates if it chose the same folder
        folder2=np.random.choice(glob.glob(file_path + "*"))
    mat2=np.zeros((480,640), dtype='float32')
    i=0
    j=0
    depth_file = np.random.choice(glob.glob(folder2 + "/*.dat"))
    with open(depth_file) as file:
        for line in file:
            vals = line.split('\t')
            for val in vals:
                if val == "\n": continue 
                if int(val) > 1200 or int(val) == -1: val= 1200
                mat2[i][j]=float(int(val))
                j+=1
                j=j%640

            i+=1
        mat2 = np.asarray(mat2)
    mat2_small=mat2[140:340,220:420]
    img2 = Image.open(depth_file[:-5] + "c.bmp")
    img2.thumbnail((640,480))
    img2 = np.asarray(img2)
    img2 = img2[140:340,220:420]
    mat2_small=(mat2_small-np.mean(mat2_small))/np.max(mat2_small)
 #   plt.imshow(img2)
 #   plt.show()
 #   plt.imshow(mat2_small)
 #   plt.show()
    full1 = np.zeros((200,200,4))
    full1[:,:,:3] = img[:,:,:3]
    full1[:,:,3] = mat_small
    
    full2 = np.zeros((200,200,4))
    full2[:,:,:3] = img2[:,:,:3]
    full2[:,:,3] = mat2_small
    return np.array([full1, full2])

In [18]:
create_wrong_rgbd("/data/blogs/keras-faceid-recognition/faceid_val/")[0].shape

(200, 200, 4)

In [19]:
%%markdown
# Network crafting
Now we create the network. We first manually create the constrative loss,
then we define the network architecture starting from the SqueezeNet architecture,
and then using it as a siamese-network for embedding faces into a manifold.
(the network for now is very big and could be heavily optimized,
but I just wanted to show a proof-of-concept)

# Network crafting
Now we create the network. We first manually create the constrative loss,
then we define the network architecture starting from the SqueezeNet architecture,
and then using it as a siamese-network for embedding faces into a manifold.
(the network for now is very big and could be heavily optimized,
but I just wanted to show a proof-of-concept)

In [20]:
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, Lambda, ELU, concatenate, GlobalAveragePooling2D, Input, BatchNormalization, SeparableConv2D, Subtract, concatenate
from keras.activations import relu, softmax
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D, AveragePooling2D
from keras.optimizers import Adam, RMSprop, SGD
from keras.regularizers import l2
from keras import backend as K

/usr/lib64/python3.4/importlib/_bootstrap.py:321: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  return f(*args, **kwds)
Using TensorFlow backend.


In [21]:
def euclidean_distance(inputs):
    assert len(inputs) == 2, \
        'Euclidean distance needs 2 inputs, %d given' % len(inputs)
    u, v = inputs
    return K.sqrt(K.sum((K.square(u - v)), axis=1, keepdims=True))
        

def contrastive_loss(y_true,y_pred):
    margin=1.
    return K.mean((1. - y_true) * K.square(y_pred) + y_true * K.square(K.maximum(margin - y_pred, 0.)))
   # return K.mean( K.square(y_pred) )

In [22]:
def fire(x, squeeze=16, expand=64):
    x = Convolution2D(squeeze, (1,1), padding='valid')(x)
    x = Activation('relu')(x)
    
    left = Convolution2D(expand, (1,1), padding='valid')(x)
    left = Activation('relu')(left)
    
    right = Convolution2D(expand, (3,3), padding='same')(x)
    right = Activation('relu')(right)
    
    x = concatenate([left, right], axis=3)
    return x

In [23]:
img_input=Input(shape=(200,200,4))

x = Convolution2D(64, (5, 5), strides=(2, 2), padding='valid')(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)

x = fire(x, squeeze=16, expand=16)

x = fire(x, squeeze=16, expand=16)

x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)


x = fire(x, squeeze=32, expand=32)

x = fire(x, squeeze=32, expand=32)

x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x)


x = fire(x, squeeze=48, expand=48)

x = fire(x, squeeze=48, expand=48)

x = fire(x, squeeze=64, expand=64)

x = fire(x, squeeze=64, expand=64)

x = Dropout(0.2)(x)

x = Convolution2D(512, (1, 1), padding='same')(x)
out = Activation('relu')(x)


modelsqueeze= Model(img_input, out)

modelsqueeze.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 200, 200, 4)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 98, 98, 64)   6464        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 98, 98, 64)   256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 98, 98, 64)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
max_poolin

In [24]:
im_in = Input(shape=(200,200,4))
#wrong = Input(shape=(200,200,3))

x1 = modelsqueeze(im_in)
#x = Convolution2D(64, (5, 5), padding='valid', strides =(2,2))(x)

#x1 = MaxPooling2D(pool_size=(3, 3), strides=(2, 2))(x1)

"""
x1 = Convolution2D(256, (3,3), padding='valid', activation="relu")(x1)
x1 = Dropout(0.4)(x1)

x1 = MaxPooling2D(pool_size=(3, 3), strides=(1, 1))(x1)

x1 = Convolution2D(256, (3,3), padding='valid', activation="relu")(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)

x1 = Convolution2D(64, (1,1), padding='same', activation="relu")(x1)
x1 = BatchNormalization()(x1)
x1 = Dropout(0.4)(x1)
"""



x1 = Flatten()(x1)

x1 = Dense(512, activation="relu")(x1)
x1 = Dropout(0.2)(x1)
#x1 = BatchNormalization()(x1)
feat_x = Dense(128, activation="linear")(x1)
feat_x = Lambda(lambda  x: K.l2_normalize(x,axis=1))(feat_x)


model_top = Model(inputs = [im_in], outputs = feat_x)

model_top.summary()

im_in1 = Input(shape=(200,200,4))
im_in2 = Input(shape=(200,200,4))

feat_x1 = model_top(im_in1)
feat_x2 = model_top(im_in2)


lambda_merge = Lambda(euclidean_distance)([feat_x1, feat_x2])


model_final = Model(inputs = [im_in1, im_in2], outputs = lambda_merge)

model_final.summary()

adam = Adam(lr=0.001)

sgd = SGD(lr=0.001, momentum=0.9)

model_final.compile(optimizer=adam, loss=contrastive_loss)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 200, 200, 4)       0         
_________________________________________________________________
model_1 (Model)              (None, 11, 11, 512)       253952    
_________________________________________________________________
flatten_1 (Flatten)          (None, 61952)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               31719936  
_________________________________________________________________
dropout_2 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
lambda_1 (Lambda)            (None, 128)               0         
Total para

In [25]:
%%markdown
# Learning phase
We write the generators that will give our model batches of data to train on, then we run the training.


# Learning phase
We write the generators that will give our model batches of data to train on, then we run the training.

In [33]:
def generator(batch_size):
  
  while 1:
    X=[]
    y=[]
    switch=True
    for _ in range(batch_size):
   #   switch += 1
      if switch:
     #   print("correct")
        X.append(create_couple_rgbd("/data/blogs/keras-faceid-recognition/faceid_train/").reshape((2,200,200,4)))
        y.append(np.array([0.]))
      else:
     #   print("wrong")
        X.append(create_wrong_rgbd("/data/blogs/keras-faceid-recognition/faceid_train/").reshape((2,200,200,4)))
        y.append(np.array([1.]))
      switch=not switch
    X = np.asarray(X)
    y = np.asarray(y)
    XX1=X[0,:]
    XX2=X[1,:]
    yield [X[:,0],X[:,1]],y


In [34]:
def val_generator(batch_size):
  
  while 1:
    X=[]
    y=[]
    switch=True
    for _ in range(batch_size):
      if switch:
        X.append(create_couple_rgbd("/data/blogs/keras-faceid-recognition/faceid_val/").reshape((2,200,200,4)))
        y.append(np.array([0.]))
      else:
        X.append(create_wrong_rgbd("/data/blogs/keras-faceid-recognition/faceid_val/").reshape((2,200,200,4)))
        y.append(np.array([1.]))
      switch=not switch
    X = np.asarray(X)
    y = np.asarray(y)
    XX1=X[0,:]
    XX2=X[1,:]
    yield [X[:,0],X[:,1]],y

In [35]:
gen = generator(16)
val_gen = val_generator(4)

In [ ]:
outputs = model_final.fit_generator(gen, steps_per_epoch=30, epochs=50, validation_data = val_gen, validation_steps=20)

Epoch 1/50
30/30 [==============================] - 707s 24s/step - loss: 0.2580 - val_loss: 0.4847
Epoch 2/50
29/30 [============================>.] - ETA: 13s - loss: 0.2490

In [ ]:
%%markdown
# Some model tests

In [ ]:
cop = create_couple("/data/blogs/keras-faceid-recognition/faceid_val/")
model_final.evaluate([cop[0].reshape((1,200,200,1)), cop[1].reshape((1,200,200,1))], np.array([0.]))

In [ ]:
cop = create_wrong_rgbd("/data/blogs/keras-faceid-recognition/faceid_val/")
model_final.predict([cop[0].reshape((1,200,200,4)), cop[1].reshape((1,200,200,4))])

In [ ]:
%%markdown
# Saving and loading the model
The next cells show both how to save the model weights
and upload them into your Drive, and then how to retrieve
those weights from the Drive to load a pre-trained model.

In [ ]:
model_final.save("/data/blogs/keras-faceid-recognition/faceid_big_rgbd_2.h5")

In [ ]:
%%markdown
# Raw output
Here we create a model that outputs the embedding of an input face
instead of the distance between two embeddings, so we can map those outputs.


In [ ]:
im_in1 = Input(shape=(200,200,4))
#im_in2 = Input(shape=(200,200,4))

feat_x1 = model_top(im_in1)
#feat_x2 = model_top(im_in2)



model_output = Model(inputs = im_in1, outputs = feat_x1)

model_output.summary()

adam = Adam(lr=0.001)

sgd = SGD(lr=0.001, momentum=0.9)

model_output.compile(optimizer=adam, loss=contrastive_loss)

In [ ]:
cop = create_couple_rgbd("/data/blogs/keras-faceid-recognition/faceid_val/")
model_output.predict(cop[0].reshape((1,200,200,4)))

In [ ]:
def create_input_rgbd(file_path):
  #  print(folder)
    mat=np.zeros((480,640), dtype='float32')
    i=0
    j=0
    depth_file = file_path
    with open(depth_file) as file:
        for line in file:
            vals = line.split('\t')
            for val in vals:
                if val == "\n": continue    
                if int(val) > 1200 or int(val) == -1: val= 1200
                mat[i][j]=float(int(val))
                j+=1
                j=j%640

            i+=1
        mat = np.asarray(mat)
    mat_small=mat[140:340,220:420]
    img = Image.open(depth_file[:-5] + "c.bmp")
    img.thumbnail((640,480))
    img = np.asarray(img)
    img = img[140:340,220:420]
    mat_small=(mat_small-np.mean(mat_small))/np.max(mat_small)
    plt.figure(figsize=(8,8))
    plt.grid(True)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(mat_small)
    plt.show()
    plt.figure(figsize=(8,8))
    plt.grid(True)
    plt.xticks([])
    plt.yticks([])
    plt.imshow(img)
    plt.show()
    
    
    
    full1 = np.zeros((200,200,4))
    full1[:,:,:3] = img[:,:,:3]
    full1[:,:,3] = mat_small
    
    return np.array([full1])

In [ ]:
%%markdown
# Data visualization
Here we store the embeddings for all the faces in the dataset.
Then, using both t-SNE and PCA, we visualize the embeddings going from 128 to 2 dimensions.

In [ ]:
outputs=[]
n=0
for folder in glob.glob('/data/blogs/keras-faceid-recognition/faceid_train/*'):
  i=0
  for file in glob.glob(folder + '/*.dat'):
    i+=1
    outputs.append(model_output.predict(create_input_rgbd(file).reshape((1,200,200,4))))
  print(i)
  n+=1
  print("Folder ", n, " of ", len(glob.glob('/data/blogs/keras-faceid-recognition/faceid_train/*')))
print(len(outputs))

In [ ]:
outputs = np.asarray(outputs)
outputs = outputs.reshape((-1,128))
outputs.shape

In [ ]:
import sklearn
from sklearn.manifold import TSNE

In [ ]:
X_embedded = TSNE(2).fit_transform(outputs)
X_embedded.shape

In [ ]:
import numpy as np
from sklearn.decomposition import PCA

X_PCA = PCA(3).fit_transform(outputs)
print(X_PCA.shape)

#X_embedded = TSNE(2).fit_transform(X_PCA)
#print(X_embedded.shape)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
color = 0
for i in range(len((X_embedded))):
  el = X_embedded[i]
  if i % 51 == 0 and not i==0:
    color+=1
    color=color%10
  plt.scatter(el[0], el[1], color="C" + str(color))

In [ ]:
%%markdown
# Distance between two arbitrary RGBD pictures

In [ ]:
file1 = ('/data/blogs/keras-faceid-recognition/faceid_train/(2012-05-16)(154211)/015_1_d.dat')
inp1 = create_input_rgbd(file1)
file1 = ('/data/blogs/keras-faceid-recognition/faceid_train/(2012-05-16)(154211)/011_1_d.dat')
inp2 = create_input_rgbd(file1)

model_final.predict([inp1, inp2])